https://yganalyst.github.io/spatial_analysis/uber_h3_keplergl/

https://dailyheumsi.tistory.com/92

https://brorica.tistory.com/148?category=949090

https://thlee33.medium.com/kepler-gl%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EA%B1%B4%EB%AC%BC-3d-%EC%8B%9C%EA%B0%81%ED%99%94-330400887fe3

https://dailyheumsi.tistory.com/147

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
from config import vworld_key
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [2]:
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])

In [3]:
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

In [4]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

### 청주시_도시재생계획구역

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/27.청주시_도시재생계획구역.geojson', encoding='utf-8') as geojson_file:
    geojson_data = json.load(geojson_file)
crp_df = pd.json_normalize(geojson_data) # city_revitalize_planning_map_df
crp_df['geometry'] = crp_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)

# 폴리곤의 중점을 찾음
centroid = crp_df['geometry'].iloc[0].centroid.buffer(0.015) # 1도의 위도 변화는 대략 111.32 킬로미터
crp_df['centroid_polygon_geometry'] = [centroid]
crp_df

,type,properties.fid,properties.구역명,geometry.type,geometry.coordinates,geometry,centroid_polygon_geometry
0,Feature,5,도시재생구역도,Polygon,"[[[127.4833417, 36.6318644], [127.4833417, 36....","POLYGON ((127.4833417 36.6318644, 127.4833417 ...",POLYGON ((127.50347965416363 36.63603755594320...


### 격자(매핑용)

In [6]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/26.청주시_격자(매핑용).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))

In [7]:
# crp_df에서 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
polygon_expanded = crp_df['centroid_polygon_geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')
#grid_map_df['geometry'] = GeoSeries(grid_map_df['geometry'])

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid = grid_map_df[grid_map_df['geometry'].intersects(polygon)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택, within 은 포함
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid_expanded = grid_map_df[grid_map_df['geometry'].intersects(polygon_expanded)].reset_index(drop=True) # polygon_expanded과 교차하거나 포함하는 경우를 모두 선택

#### 청주시 상세 도로망 & 도로명주소 데이터 필터링 및 통합

In [8]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/13.청주시_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : LineString(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadsystem = roadsystem_df[roadsystem_df['geometry'].intersects(polygon)].reset_index(drop=True)

# GeoJSON 파일 불러오기
with open('SBJ_2309_001/16.청주시_도로명주소(도로).geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_road_df = pd.json_normalize(geojson_data['features'])
roadname_road_df['geometry'] = roadname_road_df['geometry.coordinates'].apply(lambda x: LineString(x))
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadname_road_df = gpd.GeoDataFrame(roadname_road_df, geometry='geometry')
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadname_road = roadname_road_df[roadname_road_df['geometry'].within(polygon)].reset_index(drop=True)

# 통합 도로망 df
tot_roadsystem = pd.concat([filtered_roadsystem[['geometry']], filtered_roadname_road[['geometry']]])

#### 청주시 시장현황 필터링

In [9]:
# 시장 데이터프레임을 GeoPandas 데이터프레임으로 변환
market_df = pd.read_csv('SBJ_2309_001/6.청주시_시장현황.csv')
market_df = geo_transform(market_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# market_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_market = market_df[market_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 청주시 주차장 현황

In [10]:
# 시장 데이터프레임을 GeoPandas 데이터프레임으로 변환
parking_df = pd.read_csv('SBJ_2309_001/7.청주시_주차장현황.csv')
parking_df = geo_transform(parking_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0].centroid.buffer(0.01)
# parking_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_parking = parking_df[parking_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 청주시 공공기관 및 주요지점 현황

In [11]:
public_df = pd.read_csv('SBJ_2309_001/25.청주시_공공기관_및_주요지점현황.csv')
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
public_df = geo_transform(public_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# public_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_public = public_df[public_df['geometry'].within(polygon)]
filtered_points_public['inst_gbn'] = filtered_points_public['inst_gbn'].replace('공공기관', '행정기관')
filtered_points_public['inst_gbn'] = filtered_points_public['inst_gbn'].replace('시군청', '행정기관')
filtered_points_public['inst_gbn'] = filtered_points_public['inst_gbn'].replace('도단위기관', '행정기관')
filtered_points_public['inst_gbn'] = filtered_points_public['inst_gbn'].replace('민간기관', '상업시설')
filtered_points_public = filtered_points_public.drop_duplicates(subset='geometry', keep='first').reset_index(drop=True)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

#### 청주시 공원 현황

In [12]:
# 시장 데이터프레임을 GeoPandas 데이터프레임으로 변환
park_df = pd.read_csv('SBJ_2309_001/8.청주시_공원현황.csv')
park_df = geo_transform(park_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# park_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_park = park_df[park_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 청주시 가로수 현황 (공공데이터포털 - 산림청_도시숲가로수관리 가로수 현황 https://www.data.go.kr/data/15120900/fileData.do)

In [13]:
'''산림청_도시숲가로수관리 가로수 현황_20221231 데이터 정제 완료 -'''
# street_tree_df = pd.read_csv('SBJ_2309_001/산림청_도시숲가로수관리 가로수 현황_20221231.csv', encoding='cp949')
# street_tree_df = street_tree_df[['시군구명', '도로구간명', '수종명', '지역X좌표', '지역Y좌표']]
# street_tree_df = street_tree_df[street_tree_df['시군구명'].str.contains('청주')].reset_index(drop=True)

# # EPSG 5186에서 EPSG 4326으로 좌표 변환을 수행하는 함수 생성
# def transform_coordinates(row):
#     in_proj = pyproj.Proj(init='epsg:5186')
#     out_proj = pyproj.Proj(init='epsg:4326')
#     lon, lat = pyproj.transform(in_proj, out_proj, row['지역X좌표'], row['지역Y좌표'])
#     return pd.Series({'lat': lat, 'lon': lon})

# # '지역X좌표'와 '지역Y좌표'를 EPSG 4326으로 변환하고 'lat' 및 'lon' 열로 이름 변경
# street_tree_df[['lat', 'lon']] = street_tree_df.apply(transform_coordinates, axis=1)
# # '지역X좌표'와 '지역Y좌표' 열을 삭제 (선택 사항)
# street_tree_df.drop(['지역X좌표', '지역Y좌표'], axis=1, inplace=True)

# 가로수 데이터프레임 로드 및 GeoPandas 변환
street_tree_df = pd.read_csv('SBJ_2309_001/29.청주시_가로수현황.csv', encoding='UTF8')
street_tree_df = geo_transform(street_tree_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_tree = street_tree_df[street_tree_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



### 청주시 카드매출 (격자매핑)

In [16]:
# KB국민카드 매출 데이터를 활용하기 보다는 총 매출 추정금액을 활용할 것
cardsales_df = pd.read_csv('SBJ_2309_001/19.청주시_카드매출_격자매핑.csv')
cardsales_df.dropna(inplace=True)
cardsales_df.reset_index(drop=True, inplace=True)
cardsales_df['est_sales'] = (cardsales_df['est_sales'] * 1000).astype(int)
cardsales_df = cardsales_df.drop('stdr_ym', axis=1) # 시기마다의 데이터 개수가 다르고 범위도 적기 때문에, 시기정보를 나타내는 column 제거

#원도심 영역 grid id 리스트 로드 및 police_report_df 필터링
grid_id = filtered_grid['properties.gid'].tolist()
filtered_cardsales = cardsales_df[cardsales_df['gid'].isin(grid_id)]

# 'est_sales' 열의 값이 0인 행을 'card_sales' 값으로 채우기
filtered_cardsales['est_sales'] = filtered_cardsales.apply(lambda row: row['card_sales'] if row['est_sales'] == 0 else row['est_sales'], axis=1)

tot_cardsales = pd.DataFrame()
for gid in tqdm(filtered_cardsales['gid'].unique()):
    subset = filtered_cardsales[filtered_cardsales['gid'] == gid]
    subset_sum = subset.groupby(['kbc_bzc_nm_1', 'kbc_bzc_nm_2', 'kbc_bzc_nm_3'])[['card_sales', 'est_sales']].sum().reset_index()
    subset_sum['gid'] = [gid] * len(subset_sum)
    tot_cardsales = pd.concat([tot_cardsales, subset_sum])

# grid df에서 gid와 grid polygon을 매핑 
map_dic = dict(zip(grid_map_df['properties.gid'], grid_map_df['geometry']))
# gid를 map_dic에 매핑하여 geometry column생성
geometry_lst = []
for _, row in tot_cardsales.iterrows():
    geometry_lst.append(map_dic[row['gid']])
tot_cardsales['geometry'] = geometry_lst

tot_cardsales

C:\Users\HJK\AppData\Local\Temp\ipykernel_22608\1938207329.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100%|██████████| 137/137 [00:00<00:00, 239.17it/s]


,kbc_bzc_nm_1,kbc_bzc_nm_2,kbc_bzc_nm_3,card_sales,est_sales,gid,geometry
0,서비스,(정기)납부/대여서비스,도서/음반대여,1104000,22483000,다바989487,POLYGON ((127.48769549878772 36.63618970133157...
1,서비스,(정기)납부/대여서비스,미디어/음향,7900000,149249000,다바989487,POLYGON ((127.48769549878772 36.63618970133157...
2,서비스,교육서비스,서예/미술학원,20080000,263385000,다바989487,POLYGON ((127.48769549878772 36.63618970133157...
3,서비스,교육서비스,자동차운전/연수,3485300,21646000,다바989487,POLYGON ((127.48769549878772 36.63618970133157...
4,서비스,교육서비스,직업훈련소,990000,48646000,다바989487,POLYGON ((127.48769549878772 36.63618970133157...
...,...,...,...,...,...,...,...
1,소매업,취미-미술,화방/화랑,88000,2099000,다바985495,POLYGON ((127.48321957078569 36.64340113658077...
0,서비스,생활편의서비스,인쇄업,217000,4352000,다바987489,POLYGON ((127.48545797797155 36.63799244545596...
1,음식,간이주점,호프/맥주,10213000,118149000,다바987489,POLYGON ((127.48545797797155 36.63799244545596...
2,음식,커피/음료,커피전문점,2220430,17581200,다바987489,POLYGON ((127.48545797797155 36.63799244545596...


In [17]:
filtered_cardsales['kbc_bzc_nm_1'].unique()

array(['서비스', '소매업', '음식', '기타'], dtype=object)

In [18]:
# 일단 기준 1로 나눠서 시각화 하고 거기서 기준 2, 3 넘어가서 분석해야할듯 너무 많다..
filtered_cardsales['kbc_bzc_nm_2'].unique()

array(['(정기)납부/대여서비스', '의료서비스', '의류', '패션잡화', '가구', '사무/문구용품', '건축/인테리어',
       '생활편의서비스', '가정용품', '차량판매', '화장품/미용', '의약/의료기기', '한식', '취미-음악',
       '가전제품', '운동/레져용품', '도매', '여가/오락서비스', '일식', '건강식품', '서적',
       '화초 및 식물 소매', '음식료품소매', '취미-미술', '중식', '패스트푸드', '양식', '커피/음료',
       '기타외국식', '종합소매', '기업', '차량부품', '간이주점', '제과/제빵', '기념품/공예품/골동품',
       '교육서비스', '기타용품', '연료소매', '(비정기)전문서비스', '분식', '떡/한과', '노점/무점포 소매',
       '유흥주점', '뷔페', '유아용품', '완구', '애완용품'], dtype=object)

##### 카드매출 시각화 - 격자매핑

In [ ]:
# 'gid' 값을 기준으로 그룹화 및 매출 액수에 따라 색상을 지정하는 함수 정의
est_sales_sum_df = pd.DataFrame()
for gid in tot_cardsales['gid'].unique():
    subset = tot_cardsales[tot_cardsales['gid'] == gid].copy()
    est_sales_sum = subset.iloc[:, 4:5].sum()
    est_sales_sum_df = pd.concat([est_sales_sum_df, est_sales_sum])
# 그룹화한 데이터프레임을 describe하여 상위 비율에 따른 색상범위 미리 설정
top_75 = est_sales_sum_df.describe().loc['75%',0]
top_50 = est_sales_sum_df.describe().loc['50%',0]
top_25 = est_sales_sum_df.describe().loc['25%',0]
def color_picker(est_sales_sum):
    if est_sales_sum > top_75:
        return 'red'
    elif top_75 > est_sales_sum >= top_50:
        return 'orange'
    elif top_50 > est_sales_sum >= top_25:
        return 'yellow'
    elif top_25 > est_sales_sum >= 0:
        return 'green'
        
# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 100px; height: 110px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "> &nbsp; 총매출 범례 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 상위 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 상위 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 상위 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 상위 100%
     </div>
     """
# map 생성
m = folium.Map(location=[36.627797, 127.511943],  zoom_start=13)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_carddata = folium.FeatureGroup(name="카드매출", overlay=False)

# 'gid' 값을 기준으로 그룹화
for gid in tqdm(tot_cardsales['gid'].unique()):
    subset = tot_cardsales[tot_cardsales['gid'] == gid].copy()
    subset['kbc_total'] = subset.iloc[:, 0] + ' - ' +  subset.iloc[:, 2]
    est_sales_sum = subset['est_sales'].sum()
    
    # 매출 데이터를 3자리마다 쉼표로 구분하고 "원"을 추가
    subset['카테고리'] = subset['kbc_total'].copy()
    subset['총매출'] = subset['est_sales'].apply(lambda x: f'{x:,}원')
        
    # subset_popup 데이터프레임을 HTML로 변환하여 popup text에 저장
    subset_popup = subset[['카테고리', '총매출']].style.hide(axis='index').set_properties(**{'border': '1px solid black'}).to_html()
    popup_text = f"총매출 - {est_sales_sum:,}원<br><br><div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{subset_popup}</div>"
    color = color_picker(est_sales_sum)
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 0.5}
    ).add_to(m_carddata).add_child(folium.Popup(popup_text, max_width=1200))
m_carddata.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))

m

##### 카드매출 시각화 - 히트맵

In [45]:
# map 생성
m = folium.Map(location=[36.627797, 127.511943],  zoom_start=13)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 도로망 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = tot_roadsystem['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 5
    })
grid_layer.add_to(m)

# 'gid' 값을 기준으로 그룹화 및 히트맵 추가
m_card = folium.FeatureGroup(name="카드매출", overlay=True)
heatmap = []
for gid in tqdm(tot_cardsales['gid'].unique()):
    heat_data = []
    subset = tot_cardsales[tot_cardsales['gid'] == gid].copy()
    est_sales_sum = subset['est_sales'].sum()
    point = subset['geometry'].iloc[0]
    heatmap.append([float(point.centroid.xy[1][0]), float(point.centroid.xy[0][0]), float(est_sales_sum)])
HeatMap(heatmap, max_opacity=0.6, radius=27).add_to(m_card)
m_card.add_to(m)

# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

m

  0%|          | 0/137 [00:00<?, ?it/s]

100%|██████████| 137/137 [00:00<00:00, 1291.85it/s]


### 청주시 인도(보도) 시각화

In [ ]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/17.청주시_인도(보도).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
sidewalk_df = pd.json_normalize(geojson_data['features'])
sidewalk_df['geometry'] = sidewalk_df['geometry.coordinates'].apply(make_lin)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
sidewalk_df = gpd.GeoDataFrame(sidewalk_df, geometry='geometry')

In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0].centroid.buffer(0.015)

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_sidewalk = sidewalk_df[sidewalk_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_sidewalk

,type,properties.UFID,properties.QUAL,properties.BYYN,properties.KIND,geometry.type,geometry.coordinates,geometry
0,Feature,100036706050A0031fb2bb5069e324009,미분류,무,인도,LineString,"[[127.48704805856802, 36.64879715261194], [127...","LINESTRING (127.48705 36.64880, 127.48703 36.6..."
1,Feature,100036706040A0031944d0191c32c411d,미분류,무,인도,LineString,"[[127.4905110148085, 36.65017086233968], [127....","LINESTRING (127.49051 36.65017, 127.49050 36.6..."
2,Feature,100036706050A0031b5e7f9c289f64455,미분류,무,인도,LineString,"[[127.48811663566829, 36.645383249505215], [12...","LINESTRING (127.48812 36.64538, 127.48809 36.6..."
3,Feature,100036706050A00319fde6787305a4ceb,미분류,무,인도,MultiLineString,"[[[127.4821726136577, 36.646733069397726], [12...","LINESTRING (127.48217 36.64673, 127.48202 36.6..."
4,Feature,100036706050A003171957b43200a469d,미분류,무,인도,LineString,"[[127.4892521501864, 36.64684313842911], [127....","LINESTRING (127.48925 36.64684, 127.48907 36.6..."
...,...,...,...,...,...,...,...,...
624,Feature,100036706060A00318412b193c1854672,미분류,유,자전거도로,LineString,"[[127.49352268082293, 36.62464723956152], [127...","LINESTRING (127.49352 36.62465, 127.49340 36.6..."
625,Feature,100036706060A00319220d74c063e442a,블록,유,자전거도로,LineString,"[[127.4940249394998, 36.624428771048855], [127...","LINESTRING (127.49402 36.62443, 127.49389 36.6..."
626,Feature,100036706060A003119b74a925b834b89,블록,유,자전거도로,LineString,"[[127.49415527886981, 36.62422537476052], [127...","LINESTRING (127.49416 36.62423, 127.49405 36.6..."
627,Feature,100036706060A0031ca0403c6c49b4b58,블록,유,자전거도로,LineString,"[[127.4895393709331, 36.62559462233704], [127....","LINESTRING (127.48954 36.62559, 127.48970 36.6..."


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_sidewalk.iterrows():
    popup_text = f"도로명:{row['properties.KIND']}"
    color = 'black' if row['properties.KIND'] == '인도' else 'blue'
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color=color: {'fillColor': color, 'color': color, 'weight': 2}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

### 청주시 상권정보 시각화

In [ ]:
trading_area_df = pd.read_csv('SBJ_2309_001/18.청주시_상권정보.csv')
trading_area_df = geo_transform(trading_area_df)
trading_area_df

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,com_lc_cd,com_lc_nm,com_mc_cd,com_mc_nm,com_sc_cd,com_sc_nm,ksic_cd,ksic_nm,adb_emd_cd,adb_emd_nm,road_address,lon,lat,geometry
0,R,학문/교육,R04,학원-어학,R04A01,학원-외국어/어학,P85502,외국어학원,4311259000,성화.개신.죽림동,"충청북도 청주시 서원구 복대로 7, (개신동)",127.446061,36.617928,POINT (127.44606 36.61793)
1,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,비알콜 음료점업,4311154500,성안동,"충청북도 청주시 상당구 대성로 38, (서운동)",127.493371,36.628602,POINT (127.49337 36.62860)
2,D,소매,D07,가정/주방/인테리어,D07A15,유리/페인트/철물건축자재,G47519,"페인트, 유리 및 기타 건설자재 소매업",4311257000,수곡1동,"충청북도 청주시 서원구 매봉로 64, (수곡동, 산남주공3차)",127.478662,36.616083,POINT (127.47866 36.61608)
3,D,소매,D15,가구소매,D15A01,일반가구소매,G47520,가구 소매업,4311231000,남이면,"충청북도 청주시 서원구 남이면 청남로 1272, (척산리)",127.440092,36.562169,POINT (127.44009 36.56217)
4,R,학문/교육,R04,학원-어학,R04A01,학원-외국어/어학,P85502,외국어학원,4311454000,율량.사천동,"충청북도 청주시 청원구 공항로 118, (율량동)",127.485764,36.667772,POINT (127.48576 36.66777)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46118,L,부동산,L01,부동산중개,L01A01,부동산중개,L68221,부동산 자문 및 중개업,4311454000,율량.사천동,"충청북도 청주시 청원구 사뜸로61번길 106, (사천동, 대창5차)",127.476366,36.664730,POINT (127.47637 36.66473)
46119,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,4311425300,오창읍,"충청북도 청주시 청원구 오창읍 두릉유리로 1130-15, (장대리)",127.455312,36.741021,POINT (127.45531 36.74102)
46120,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,4311154500,성안동,"충청북도 청주시 상당구 무심동로390번길 15, (서문동)",127.484757,36.634341,POINT (127.48476 36.63434)
46121,D,소매,D02,선물/팬시/기념품,D02A02,꽃집/꽃배달,G47851,화초 및 산식물 소매업,4311374700,가경동,"충청북도 청주시 흥덕구 풍년로198번길 56-1, (가경동)",127.432312,36.631408,POINT (127.43231 36.63141)


['음식', '소매', '생활서비스', '학문/교육', '부동산', '관광/여가/오락', '숙박', '스포츠'],

In [ ]:
# a = filtered_trading_area[filtered_trading_area['com_lc_nm'] == '소매']
# a['com_mc_nm'].unique()

In [ ]:
# filtered_trading_area['com_lc_nm'].value_counts()

In [ ]:
# a['com_mc_nm'].value_counts()

In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_trading_area = trading_area_df[trading_area_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_trading_area

,com_lc_cd,com_lc_nm,com_mc_cd,com_mc_nm,com_sc_cd,com_sc_nm,ksic_cd,ksic_nm,adb_emd_cd,adb_emd_nm,road_address,lon,lat,geometry
0,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,기타 주점업,4311154500,성안동,"충청북도 청주시 상당구 남사로140번길 6, (남문로2가)",127.490409,36.631851,POINT (127.49041 36.63185)
1,D,소매,D10,건강/미용식품,D10A07,건강원,G47216,건강보조식품 소매업,4311154500,성안동,"충청북도 청주시 상당구 상당로3번길 18-1, (석교동)",127.490050,36.628551,POINT (127.49005 36.62855)
2,D,소매,D05,의복의류,D05A10,한복/갈옷/민속옷,G47412,한복 소매업,4311154500,성안동,"충청북도 청주시 상당구 무심동로336번길 100-2, (남문로1가)",127.488456,36.631302,POINT (127.48846 36.63130)
3,D,소매,D05,의복의류,D05A02,캐쥬얼/스포츠의류,G47416,셔츠 및 기타 의복 소매업,4311154500,성안동,"충청북도 청주시 상당구 상당로 55, (남문로2가)",127.490343,36.633284,POINT (127.49034 36.63328)
4,D,소매,D07,가정/주방/인테리어,D07A02,주방용품,G47592,식탁 및 주방용품 소매업,4311154500,성안동,"충청북도 청주시 상당구 상당로1번길 22, (석교동)",127.489694,36.628361,POINT (127.48969 36.62836)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,D,소매,D23,자동차/자동차용품,D23A11,자동차유리전문점,G45219,기타 자동차신품 부품 및 내장품 판매업,4311152500,중앙동,"충청북도 청주시 상당구 사북로 138, (영동)",127.486152,36.644819,POINT (127.48615 36.64482)
3174,F,생활서비스,F05,인력/고용/용역알선,F05A02,인력공급/고용알선,NaN,NaN,4311154500,성안동,"충청북도 청주시 상당구 용담로 7, (문화동, 문화동센트럴칸타빌)",127.492115,36.632598,POINT (127.49211 36.63260)
3175,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,4311154500,성안동,"충청북도 청주시 상당구 사직대로362번길 46, (서문동)",127.487471,36.634714,POINT (127.48747 36.63471)
3176,D,소매,D16,화장품소매,D16A01,화장품판매점,G47813,화장품 및 방향제 소매업,4311152500,중앙동,"충청북도 청주시 상당구 상당로 127, (북문로2가)",127.489698,36.639536,POINT (127.48970 36.63954)


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_trading_area.iterrows():
    popup_text = f"{row['com_lc_nm']}-{row['com_mc_nm']}-{row['com_sc_nm']}"
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m